In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd
import dask.distributed
import dask_jobqueue
import numpy as np
import os
import pandas as pd
import pathlib
import plotly.express as px

In [ ]:
from smc01.postprocessing.transform import DataframeToExample, NormalizeDataframe

## Load dataset in memory

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
INPUT_DATASET = DATA_DIR / 'interpolated/2021-12-20-gdps-metar/'

In [ ]:
input_path = pathlib.Path(INPUT_DATASET)

In [ ]:
sample_path = next(iter(input_path.glob('*.parquet')))

In [ ]:
sample = pd.read_parquet(sample_path)

In [ ]:
columns = set(sample.columns)
columns -= set(['gdps_hpbl'])

In [ ]:
(sample['step'] / (3*3600)).astype(int).to_numpy(copy=True)

In [ ]:
t1 = NormalizeDataframe()

In [ ]:
t1(sample)['step']

In [ ]:
t = DataframeToExample(DATA_DIR / 'interpolated/2021-12-20-gdps-metar/stations.csv')

In [ ]:
t(sample)

gdps_hpbl is partially missing from the dataset which would cause problems if we don't remove it.
To be fix in the dataset interpolation eventually.

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=4)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
df = dd.read_parquet(sorted(list(input_path.glob('*.parquet'))), columns=columns)

In [ ]:
stations = sorted(list(df['station'].unique().compute()))

In [ ]:
with open('stations.csv', 'w') as f:
    f.writelines([s + '\n' for s in stations])